In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=ea230cc17e3b4ddb9172d403e8382032 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=5bdd95a8288148658d406d7f207f8c74 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:588: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.21it/s]

Uploading: numpy-data
syft_node_location=None syft_client_verify_key=None output={'id': <UID: ca79a93f5c4f4aacb6834d417bbf3dcc>, 'name': 'numpy-data', 'description': syft.service.dataset.dataset.MarkdownDescription, 'contributors': {syft.service.dataset.dataset.Contributor}, 'data_subjects': [], 'node_uid': <UID: 8212e6797fde4c3fba4fc53ab555a886>, 'action_id': <UID: fd539f070c9c4496b227e93b5eaf90d5>, 'data': array([15, 16, 17, 18, 19]), 'mock': array([10, 11, 12, 13, 14]), 'shape': (5,), 'mock_is_real': True, 'created_at': syft.types.datetime.DateTime, 'uploader': syft.service.dataset.dataset.Contributor} node=Domain: test_h - 8212e6797fde4c3fba4fc53ab555a886 - domain

Services:
ActionService, BlobStorageService, CodeHistoryService, DataSubjectMemberService, DataSubjectService, DatasetService, EnclaveService, JobService, LogService, MetadataService, MigrateStateService, NetworkService, NotificationService, NotifierService, OutputService, PolicyService, ProjectService, QueueService, Req

SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.62it/s]


syft_node_location=None syft_client_verify_key=None output={'id': <UID: bf9da0e2935e4a31903b9f20d5fb0d3e>, 'name': 'numpy-data', 'description': syft.service.dataset.dataset.MarkdownDescription, 'contributors': {syft.service.dataset.dataset.Contributor}, 'data_subjects': [], 'node_uid': <UID: 8a1c04544655402190588aec30079bc3>, 'action_id': <UID: a924e4f0cc8143b28c530f5c0340a4a7>, 'data': Pointer:
None, 'mock': array([0, 1, 2, 3, 4]), 'shape': (5,), 'mock_is_real': True, 'created_at': syft.types.datetime.DateTime, 'uploader': syft.service.dataset.dataset.Contributor} node=Domain: test_l - 8a1c04544655402190588aec30079bc3 - domain

Services:
ActionService, BlobStorageService, CodeHistoryService, DataSubjectMemberService, DataSubjectService, DatasetService, EnclaveService, JobService, LogService, MetadataService, MigrateStateService, NetworkService, NotificationService, NotifierService, OutputService, PolicyService, ProjectService, QueueService, RequestService, SettingsService, SyftImageRe

In [8]:
dataset_storage_permissions = node_low.python_node.get_service(
    "datasetservice"
).stash.partition.storage_permissions
dataset_id = client_low.datasets[0].id

assert dataset_storage_permissions[dataset_id] == {node_low.python_node.id}

# Data Scientist: make requests

In [9]:
@sy.syft_function()
def sub_function(data) -> float:
    print("executing sub function")

    return data.mean()


client_low_ds.code.submit(sub_function)

SyftSuccess: Syft function 'sub_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: User Code Submitted

In [10]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(domain, data) -> float:
    job = domain.launch_job(sub_function, data=data)
    return job


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/d89048e5d2ef4c05a170c556c5313a7c.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=02a7af2e0ba44f3496c8070214499c91 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_4954: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_4954
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
```python
class Job:
    id: UID = 589930bb67f74c9d9800157facae3e7a
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [11]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [12]:
client_low_ds.code.compute_mean(data=data_low)

SyftError: Execution denied, Please wait for the code to be approved

## Data Owner: Sync to high side

In [13]:
client_low.code[1].nested_codes.values()

SyftInfo: Node Landed!


dict_values([(syft.store.linked_obj.LinkedObject, {})])

In [14]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [15]:
low_state

syft.service.sync.sync_state.SyncState

In [16]:
assert (
    set(low_state.objects.keys())
    == set(low_state.permissions.keys())
    == set(low_state.storage_permissions.keys())
)

In [17]:
diff_state = compare_states(low_state, high_state)

In [18]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [19]:
diff_state.diffs[1].low_obj.get_sync_dependencies()

[]

In [20]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #8327641b26b840d8b9d07f91f8dc604b:
service_func_name: sub_function
input_owners: [
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
worker_pool_name: default-pool

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #710a26105cf14a2eaf5e1725c3c88708:
    approved: False
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

No high side changes.

Decision: Syncing 2 objects from low side


LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #b71ef09265d14cc184aed007dff8563d:
service_func_name: compu

In [21]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, syft.service.request.request.Request],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [22]:
assert len(resolved_state_high.new_permissions) == 0

new_objs = resolved_state_high.create_objs
assert {o.id for o in new_objs} == {
    sp.uid for sp in resolved_state_high.new_storage_permissions
}

In [23]:
client_low.apply_state(resolved_state_low)

/Users/eelco/dev/PySyft/packages/syft/src/syft/types/syft_object.py:588: TypeHintWarning: Skipping type check against 'ServiceRole'; this looks like a string-form forward reference imported from another module
  check_type(value, var_annotation)


SyftSuccess: Synced 0 items

In [24]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 5 items

In [25]:
request_storage_permissions = node_high.python_node.get_service(
    "requestservice"
).stash.partition.storage_permissions
request_id = client_high.requests[0].id
node_high_id = node_high.python_node.id

assert node_high_id in request_storage_permissions[request_id]

# Data Owner: Run code high-side

In [26]:
data_high = client_high.datasets[0].assets[0]

In [27]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 132efd4d31ca41f6b773ac8e42d7d88d
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [28]:
# wait for the result
job_high.wait().get()

13/03/24 12:11:22 FUNCTION LOG (37ec0db5384b47d898713fd1ade8ac7e): executing sub function


```python
class Job:
    id: UID = 37ec0db5384b47d898713fd1ade8ac7e
    status: completed
    has_parent: True
    result: 17.0
    logs:

0 executing sub function
JOB COMPLETED
    
```

In [29]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [30]:
job_info

syft.service.job.job_stash.JobInfo

In [31]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [32]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: ea75f59a96514dc1b6f6176e602ddeaa as e0eac5b4418f478e356894de4b3dd3b8232d1941a03c16263d05dbfa41213fcd] ea75f59a96514dc1b6f6176e602ddeaa
Job(132efd4d31ca41f6b773ac8e42d7d88d) Setting new result ea75f59a96514dc1b6f6176e602ddeaa -> ea75f59a96514dc1b6f6176e602ddeaa


SyftSuccess: Request submitted for updating result.

In [33]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [34]:
assert action_store_high.storage_permissions[job_high.result.id.id] == {
    node_high.python_node.id
}

## Data Owner: Sync back to low-side

In [35]:
node_high.python_node.get_role_for_credentials(client_high.verify_key)

<ServiceRole.ADMIN: 128>

In [36]:
node_high.python_node.get_service("userservice").stash.partition.data

{<UID: 2e4e9372330744198341b72f703d3c53>: syft.service.user.user.User}

In [37]:
client_high.verify_key

d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d

In [38]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [39]:
diff_state_2 = compare_states(low_state, high_state)

In [40]:
for i, diff in enumerate(diff_state_2.diffs):
    print(i, diff.obj_type, diff.low_obj, diff.high_obj)

0 <class 'syft.service.code.user_code.UserCodeStatusCollection'> {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')} {NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.PENDING: 'pending'>, '')}
1 <class 'syft.service.code.user_code.UserCode'> syft.service.code.user_code.UserCode syft.service.code.user_code.UserCode
2 <class 'syft.service.code.user_code.UserCodeStatusCollection'> {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')} {NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}
3 <class 'syft.service.code.user_code.UserCode'> syft.service.code.user_code.UserCode syft.service.code.user_code.UserCode
4 <class 'syft.service.request.request.Request'> syft.service.request.request.Request syft.service.request.request.Request
5 <class 'syft.service.output.output_service.ExecutionOutput'> None syft.service.output.output_service.Executio

In [41]:
diff_state_2.diffs[6].high_obj.__repr_attrs__

[]

In [42]:
resolved_state_low, resolved_state_high = resolve(diff_state_2)

LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #8327641b26b840d8b9d07f91f8dc604b:
service_func_name: sub_function
input_owners: [
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #710a26105cf14a2eaf5e1725c3c88708:
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #8327641b26b840d8b9d07f91f8dc604b:
service_func_name: sub_function
input_owners: [
]
code_status: [
‎ ‎ ‎ ‎ Node: test_h, Status: pending
]

    ―――― DIFF ―――――――――――――――――――――――――――――――――――――――

 high


Decision: Syncing 2 objects from high side


LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #b71ef09265d14cc184aed007dff8563d:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #5ffe00e936b8480b889d64de73b7284b:
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #b71ef09265d14cc184aed007dff8563d:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_

 high


ValueError: too many user codes

In [ ]:
print(resolved_state_low)
print()
print(resolved_state_high)

In [ ]:
res = client_low.apply_state(resolved_state_low)
res

In [ ]:
client_high.apply_state(resolved_state_high)

In [ ]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [ ]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

In [ ]:
client_low.code.compute_mean.jobs[0].log_id

In [ ]:
node_low.python_node.get_service("logservice").stash.partition.data[
    client_low.code.compute_mean.jobs[0].log_id
]

In [ ]:
node_low.python_node.get_service("logservice").stash.partition.storage_permissions

# Run code low

## Run

In [ ]:
client_low_ds._fetch_api(client_low.credentials)

In [ ]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

In [ ]:
res_low.id == job_high.result.id

In [ ]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [ ]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

In [ ]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [ ]:
private_high.mean()

In [ ]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

In [ ]:
job_low.wait().get()

In [ ]:
job_low.logs()

In [ ]:
job_high.result.id

In [ ]:
job_high.result.as_empty().id

In [ ]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)

In [ ]:
job_high.result.id

In [ ]:
job_low.result.id

In [ ]:
job_high